# Problem Set #7

## Tc2037 Implementation of Computational Methods

March 11, 2021.

_Authors of this notebook’s solution:_

- _Student ID and Name:_
- _Student ID and Name:_

**Instructions:** Write the requested Clojure code. Make sure each function passes all the unit tests.

In [1]:
;; External function required for this notebook.
(require '[clojure.test :refer [is]])

nil

# Symbolic Differentiation
---

## Introduction

As an illustration of symbol manipulation and data abstraction, consider the design of a function that performs symbolic differentiation of algebraic expressions. We would like the function to take as arguments an algebraic expression and a variable and to return the derivative of the expression with respect to the variable. For example, if the arguments to the function are $ax^2+bx+c$ and $x$, the function should return $2ax+b$. Symbolic differentiation is of special historical significance in Lisp. It was one of the motivating examples behind the development of a computer language for symbol manipulation.

In developing the symbolic-differentiation program, we will follow a **data abstraction** strategy. That is, we will first define a differentiation algorithm that operates on abstract objects such as “sums”, “products”, and “variables” without worrying about how these are represented. Only afterward will we address the representation problem.

## Basic Program

We want a Clojure function `(deriv exp variable)` that allows doing symbolic differentiation using the following reduction rules:

### Derivative With Respect To Itself

$$
\frac{d(x)}{dx} = 1
$$

### Constant Rule

$$
\frac{d(c)}{dx} = 0 
$$

Where $c$ is a constant or a variable different from $x$.

### Sum Rule

$$
\frac{d(u+v)}{dx} = \frac{d(u)}{dx} + \frac{d(v)}{dx}\
$$

### Product Rule
$$
\frac{d(u \cdot v)}{dx} = u \cdot \left ( \frac{d(v)}{dx} \right ) + v \cdot \left ( \frac{d(u)}{dx} \right )
$$

We’ll use (and later define) the following helper functions:

* `(variable? exp)` : Is `exp` a variable?
* `(same-variable? v1 v2)` : Are `v1` and `v2` the same variable?
* `(sum? exp)` : Is `exp` a sum?
* `(augend exp)` : Augend of the sum `exp`.
* `(addend exp)` : Addend of the sum `exp`.
* `(make-sum exp1 exp2)` : Construct the sum of `exp1` and `exp2`.
* `(product? exp)` : Is `exp` a product?
* `(multiplicand exp)` : Multiplicand of the product `exp`.
* `(multiplier exp)` : Multiplier of the product `exp`.
* `(make-product exp1 exp2)` : Construct the product of `exp1` and `exp2`.

In [2]:
(declare variable? same-variable? sum? augend addend make-sum 
         product? multiplicand multiplier make-product)

#'user/make-product

### `deriv` Function

In [3]:
(defn deriv
  [exp variable]
  (cond
    (number? exp)
    0
    
    (variable? exp)
    (if (same-variable? exp variable) 1 0)
    
    (sum? exp)
    (let [u (augend exp)
          v (addend exp)]
      (make-sum (deriv u variable)
                (deriv v variable)))
    
    (product? exp)
    (let [u (multiplicand exp)
          v (multiplier exp)]
      (make-sum (make-product u (deriv v variable))
                (make-product v (deriv u variable))))))

#'user/deriv

### `variable?` Function

`(variable? exp)` : Is `exp` a variable?

In [4]:
(defn variable?
  [exp]
  (symbol? exp))

#'user/variable?

In [5]:
;;; Unit tests:
(is (variable? 'x))
(is (not (variable? 42)))

true

### `same-variable?` Function

`(same-variable? v1 v2)` : Are `v1` and `v2` the same variable?

In [6]:
(defn same-variable? 
  [v1 v2]
  (and (variable? v1)
       (variable? v2)
       (= v1 v2)))

#'user/same-variable?

In [7]:
;;; Unit tests:
(is (same-variable? 'x 'x))
(is (not (same-variable? 'x 'y)))
(is (not (same-variable? 42 42)))
(is (not (same-variable? 'x 42)))

true

### `sum?` Function

`(sum? exp)` : Is `exp` a sum?

In [8]:
(defn sum? 
  [exp]
  (and (list? exp)
       (= 3 (count exp))
       (= '+ (first exp))))

#'user/sum?

In [9]:
;;; Unit tests:
(is (sum? '(+ x 2)))
(is (not (sum? 'x)))
(is (not (sum? '(+ x y z))))
(is (not (sum? '(* x y))))

true

### `augend` Function

`(augend exp)` : Augend of the sum `exp`.

In [10]:
(defn augend 
  [exp]
  (nth exp 1))

#'user/augend

In [11]:
;;; Unit test:
(is (= 'x (augend '(+ x 1))))

true

### `addend` Function

`(addend exp)` : Addend of the sum `exp`.

In [12]:
(defn addend 
  [exp]
  (nth exp 2))

#'user/addend

In [13]:
;;; Unit test:
(is (= 1 (addend '(+ x 1))))

true

### `make-sum` Function

`(make-sum exp1 exp2)` : Construct the sum of `exp1` and `exp2`.

In [14]:
(defn make-sum 
  [exp1 exp2]
  (list '+ exp1 exp2))

#'user/make-sum

In [15]:
;;; Unit tests:
(is (= '(+ x 1) (make-sum 'x 1)))
(is (= '(+ 2 2) (make-sum  2 2)))

true

### `product?` Function

`(product? exp)` : Is `exp` a product?

In [16]:
(defn product? 
  [exp]
  (and (list? exp)
       (= 3 (count exp))
       (= '* (first exp))))

#'user/product?

In [17]:
;;; Unit tests:
(is (product? '(* x 2)))
(is (not (product? 'x)))
(is (not (product? '(* x y z))))
(is (not (product? '(+ x y))))

true

### `multiplicand` Function

`(multiplicand exp)` : Multiplicand of the product `exp`.

In [18]:
(defn multiplicand 
  [exp]
  (nth exp 1))

#'user/multiplicand

In [19]:
;;; Unit test:
(is (= 'x (multiplicand '(* x 2))))

true

### `multiplier` Function

`(multiplier exp)` : Multiplier of the product `exp`.

In [20]:
(defn multiplier
  [exp]
  (nth exp 2))

#'user/multiplier

In [21]:
;;; Unit test:
(is (= 2 (multiplier '(* x 2))))

true

### `make-product` Function

`(make-product exp1 exp2)` : Construct the product of `exp1` and `exp2`.

In [22]:
(defn make-product
  [exp1 exp2]
  (list '* exp1 exp2))

#'user/make-product

In [23]:
;;; Unit tests:
(is (= '(* x 2) (make-product 'x 2)))
(is (= '(* 3 2) (make-product 3 2)))

true

### Testing It All

The following tests check that:

$$
\begin{align*}
\frac{d(x + 3)}{dx} 
&= \frac{d(x)}{dx} + \frac{d(3)}{dx} \\
&= 1 + 0
\end{align*}
$$

$$
\begin{align*}
\frac{d(x \cdot y)}{dx} 
&= x \cdot \left ( \frac{d(y)}{dx} \right ) + y \cdot \left ( \frac{d(x)}{dx} \right ) \\
&= (x \cdot 0) + (y \cdot 1)
\end{align*}
$$

$$
\begin{align*}
\frac{d([x \cdot y] \cdot [x + 3])}{dx}
&= [x \cdot y] \cdot \left ( \frac{d(x + 3)}{dx} \right ) + [x + 3] \cdot \left ( \frac{d(x \cdot y)}{dx} \right ) \\
&= ([x \cdot y] \cdot [1 + 0]) + \{ [x + 3] \cdot ([x \cdot 0] + [y \cdot 1]) \}
\end{align*}
$$

In [24]:
;;; Unit tests:
(is (= '(+ 1 0) 
       (deriv '(+ x 3) 'x)))
(is (= '(+ (* x 0) (* y 1)) 
       (deriv '(* x y) 'x)))
(is (= '(+ (* (* x y) (+ 1 0)) (* (+ x 3) (+ (* x 0) (* y 1))))
       (deriv '(* (* x y) (+ x 3)) 'x)))

true

## Modification #1

Modify the `make-sum` function in order to reduce its answers to its simplest form:

* $x + 0 = x$
* $0 + x = x$

In the same manner, modify the `make-product` function in order to reduce its answers to its simplest form:

* $x \cdot 0 = 0$
* $0 \cdot x = 0$
* $x \cdot 1 = x$
* $1 \cdot x = x$

### Modified `make-sum` Function

`(make-sum exp1 exp2)` : Construct the sum of `exp1` and `exp2`, and simplify the result.

In [25]:
(defn make-sum 
  [exp1 exp2]
  (cond
    (= exp2 0) exp1
    (= exp1 0) exp2
    (and (number? exp1) (number? exp2)) (+ exp1 exp2)
    :else (list '+ exp1 exp2)))

#'user/make-sum

In [26]:
;;; Unit tests:
(is (= 'x (make-sum 'x 0)))
(is (= 'x (make-sum 0 'x)))
(is (= 13 (make-sum 6 7)))
(is (= '(+ x 5) (make-sum 'x 5)))

true

### Modified `make-product` Function

`(make-product exp1 exp2)` : Construct the product of `exp1` and `exp2`, and simplify the result.

In [27]:
(defn make-product
  [exp1 exp2]
  (cond
    (or (= exp1 0) (= exp2 0)) 0
    (= exp1 1) exp2
    (= exp2 1) exp1
    (and (number? exp1) (number? exp2)) (* exp1 exp2)
    :else (list '* exp1 exp2)))

#'user/make-product

In [28]:
;;; Unit tests:
(is (= 0 (make-product 'x 0)))
(is (= 0 (make-product 0 'x)))
(is (= 'x (make-product 'x 1)))
(is (= 'x (make-product 1 'x)))
(is (= 30 (make-product 5 6)))
(is (= '(* x 2) (make-product 'x 2)))

true

### Testing It All

Here’s how this new version should work on the three previous examples:

$$
\begin{align*}
\frac{d(x + 3)}{dx} 
&= \frac{d(x)}{dx} + \frac{d(3)}{dx} \\
&= 1 + 0 \\
&= 1
\end{align*}
$$

$$
\begin{align*}
\frac{d(x \cdot y)}{dx} 
&= x \cdot \left ( \frac{d(y)}{dx} \right ) + y \cdot \left ( \frac{d(x)}{dx} \right ) \\
&= (x \cdot 0) + (y \cdot 1) \\
&= y
\end{align*}
$$

$$
\begin{align*}
\frac{d([x \cdot y] \cdot [x + 3])}{dx}
&= [x \cdot y] \cdot \left ( \frac{d(x + 3)}{dx} \right ) + [x + 3] \cdot \left ( \frac{d(x \cdot y)}{dx} \right ) \\
&= ([x \cdot y] \cdot [1 + 0]) + \{ [x + 3] \cdot ([x \cdot 0] + [y \cdot 1]) \} \\
&= [x \cdot y] \cdot([x + 3] \cdot y)
\end{align*}
$$

In [29]:
;;; Unit tests:
(is (= 1 (deriv '(+ x 3) 'x)))
(is (= 'y (deriv '(* x y) 'x)))
(is (= '(+ (* x y) (* (+ x 3) y)) 
       (deriv '(* (* x y) (+ x 3)) 'x)))

true

## Modification #2
Extend the basic program in order to implement the following differentiation rule:

### Power Rule

$$
\frac{d(u^v)}{dx} = (v \cdot u^{v-1}) \cdot \left ( \frac{du}{dx} \right )
$$

Add a new clause to the `deriv` function and define appropriate functions `exponentiation?`, `base`, `exponent`, and `make-exponentiation`. Use the symbol `**` to denote exponentiation.

In [30]:
(declare exponentiation? base exponent make-exponentiation)

#'user/make-exponentiation

### Modified `deriv` Function

In [31]:
(defn deriv
  [exp variable]
  (cond
    (number? exp)
    0
    
    (variable? exp)
    (if (same-variable? exp variable) 1 0)
    
    (sum? exp)
    (let [u (augend exp)
          v (addend exp)]
      (make-sum (deriv u variable)
                (deriv v variable)))
    
    (product? exp)
    (let [u (multiplicand exp)
          v (multiplier exp)]
      (make-sum (make-product u (deriv v variable))
                (make-product v (deriv u variable))))
    
    ;;; New code goes here
    ))

#'user/deriv

### `exponentiation?` Function

`(exponentiation? exp)` : Is `exp` an exponentiation?

In [32]:
(defn exponentiation? 
  [exp]
  ;;; your code goes here
  nil)

#'user/exponentiation?

In [ ]:
;;; Unit tests:
(is (exponentiation? '(** x 2)))
(is (not (exponentiation? 'x)))
(is (not (exponentiation? '(** x y z))))
(is (not (exponentiation? '(+ x y))))

### `base` Function

`(base exp)` : Base of the power `exp`.

In [34]:
(defn base 
  [exp]
  ;;; your code goes here
  nil)

#'user/base

In [ ]:
;;; Unit test:
(is (= 'x (base '(** x 2))))

### `exponent` Function

`(exponent exp)` : Exponent of the power `exp`.

In [36]:
(defn exponent 
  [exp]
  ;;; your code goes here
  nil)

#'user/exponent

In [ ]:
;;; Unit test:
(is (= 2 (exponent '(** x 2))))

### `make-exponent` Function

`(make-exponent exp1 exp2)` : Construct the power of `exp1` and `exp2`. Use the symbol `**` to denote exponentiation. If possible, simplify the result as follows:

* $x^0 = 1$
* $x^1 = x$
* $0^x = 0$
* $1^x = 1$
* If both the base and the exponent are numbers, the corresponding operation should be carried out.

In [38]:
;; Import the expt function from clojure.math.numeric-tower
(require '[cemerick.pomegranate :refer [add-dependencies]])
(add-dependencies :coordinates '[[org.clojure/math.numeric-tower "0.0.4"]])
(require '[clojure.math.numeric-tower :refer [expt]])

nil

In [39]:
(defn make-exponentiation
  [exp1 exp2]
  ;;; your code goes here
  nil)

#'user/make-exponentiation

In [ ]:
;;; Unit tests:
(is (= '(** x 2) (make-exponentiation 'x 2)))
(is (= 1 (make-exponentiation 'x 0)))
(is (= 'x (make-exponentiation 'x 1)))
(is (= 0 (make-exponentiation 0 'x)))
(is (= 1 (make-exponentiation 1 'x)))
(is (= 8 (make-exponentiation 2 3)))

### Testing It All

The following tests check that:

$$
\begin{align*}
\frac{d(x^2)}{dx} 
&= (2 \cdot x^{2 - 1}) \cdot \left ( \frac{d(x)}{dx} \right ) \\
&= (2 \cdot x^1) \cdot 1 \\
&= 2 \cdot x
\end{align*}
$$

$$
\begin{align*}
\frac{d(2^x)}{dx} 
&= (x \cdot 2^{x - 1}) \cdot \left ( \frac{d(2)}{dx} \right ) \\
&= (x \cdot 2^{x - 1}) \cdot 0 \\
&= 0
\end{align*}
$$

$$
\begin{align*}
\frac{d([x + 3]^y)}{dx} 
&= (y \cdot [x + 3]^{y - 1}) \cdot  \left ( \frac{d(x + 3)}{dx} \right ) \\
&= (y \cdot [x + 3]^{y - 1}) \cdot  \left ( \frac{d(x)}{dx} + \frac{d(3)}{dx} \right ) \\
&= (y \cdot [x + 3]^{y - 1}) \cdot (1 + 0) \\
&= y \cdot (x + 3)^{y - 1}
\end{align*} 
$$

$$
\begin{align*}
\frac{d([x \cdot y]^x)}{dx} 
&= (x \cdot [x \cdot y]^{x - 1}) \cdot \left ( \frac{d(x \cdot y)}{dx} \right ) \\
&= (x \cdot [x \cdot y]^{x - 1}) \cdot \left [ x \cdot \left ( \frac{d(y)}{dx} \right ) + y \cdot \left ( \frac{d(x)}{dx} \right ) \right ]\\
&= (x \cdot [x \cdot y]^{x - 1}) \cdot [ (x \cdot 0) + (y \cdot 1) ] \\
&= (x \cdot [x \cdot y]^{x - 1}) \cdot y
\end{align*} 
$$

In [ ]:
;;; Unit tests:
(is (= '(* 2 x) 
       (deriv '(** x 2) 'x)))
(is (= 0
       (deriv '(** 2 x) 'x)))
(is (= '(* y (** (+ x 3) (+ y -1)))
       (deriv '(** (+ x 3) y) 'x)))
(is (= '(* (* x (** (* x y) (+ x -1))) y)
       (deriv '(** (* x y) x) 'x)))

## Credits 

This exercise was taken from chapter 2, section [2.3.2](https://mitpress.mit.edu/sites/default/files/sicp/full-text/book/book-Z-H-16.html#%_sec_2.3.2), of “Structure and Interpretation of Computer Programs”, second edition, by Harold Abelson, Gerald Jay Sussman, and Julie Sussman, published by The MIT Press.